In [ ]:
import random

In [ ]:

# Function to generate CV
def generate_cv(bio, include_gender=True):
    cv = f"Name: {bio['name']}\n"
    if include_gender:
        cv += f"Gender: {bio['gender']}\n"
    cv += f"Email: {bio['contact_info']['email']}\n"
    cv += f"Phone: {bio['contact_info']['phone']}\n\n"
    cv += f"Education: {bio['education']}\n"
    cv += f"Skills: {', '.join(bio['skills'])}\n"
    cv += "Experience:\n"
    for exp in bio['experience']:
        cv += f"- {exp['title']} at {exp['company']} ({exp['years']} years)\n"
    return cv

In [ ]:
# Function to generate Cover Letter
def generate_cover_letter(bio, job, include_gender=True):
    intro = f"Dear Hiring Manager at {job['company']},\n\n"
    if include_gender and bio['gender'] == "Female":
        intro = intro.replace("Hiring Manager", "Dear Sir/Madam")
    intro += f"I am writing to express my interest in the {job['title']} position at {job['company']}. With my background in computer science and experience as a software developer and data analyst, I believe I would be a strong fit for your team.\n\n"
    
    skills = " ".join([f"My skills in {skill}" for skill in bio['skills']])
    cover_letter = intro + f"{skills} make me well-equipped to contribute to your team.\n"
    
    cover_letter += "\nThank you for considering my application. I look forward to the opportunity to discuss how my skills can contribute to your team’s success.\n\nSincerely,\n"
    cover_letter += f"{bio['name']}"
    
    return cover_letter


In [ ]:

# Function to generate both CV and Cover Letter
def generate_application(bio, job_ad, include_gender=True):
    cv = generate_cv(bio, include_gender)
    cover_letter = generate_cover_letter(bio, job_ad, include_gender)
    return cv, cover_letter

# Example usage
include_gender = True  # Set to False to exclude gender
cv, cover_letter = generate_application(sample_bios, job_ad, include_gender)

# Output the generated CV and Cover Letter
print("Generated CV:\n", cv)
print("\nGenerated Cover Letter:\n", cover_letter)


In [ ]:
import pandas as pd
import torch
from transformers import pipeline

# Load the CSV file containing sample bios
bio_df = pd.read_csv("sample_bios.csv")  # Update with the actual file path

# Read the job ad prompt from a text file
with open("job_ad_prompt.txt", "r") as f:
    job_ad_prompt = f.read().strip()

# Load the LLaMA model with memory optimization
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",  # Change if needed
    torch_dtype=torch.float16,  # Use FP16 to reduce memory
    device_map="auto"  # Automatically allocate to available GPU/CPU
)

# Generate text for each bio
generated_texts = []
for bio in bio_df["bio"]:  # Assuming 'bio' is the column name in the CSV
    input_text = f"{job_ad_prompt}\n\n{bio}"  # Combine job ad and bio
    output = pipe(input_text, max_length=512, num_return_sequences=1)
    generated_texts.append(output[0]["generated_text"])

# Save results to a new CSV file
bio_df["generated_text"] = generated_texts
bio_df.to_csv("generated_responses.csv", index=False)

print("Generated responses saved to generated_responses.csv")
